In [1]:
#www.kaggle.com/competitions/playground-series-s4e11/overview/$citation

In [2]:
#pip install xgboost

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: xgboost in c:\users\lucas\appdata\local\packages\pythonsoftwarefoundation.python.3.10_qbz5n2kfra8p0\localcache\local-packages\python310\site-packages (2.1.2)




[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\lucas\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
#pip install nbstripout

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: nbstripout in c:\users\lucas\appdata\local\packages\pythonsoftwarefoundation.python.3.10_qbz5n2kfra8p0\localcache\local-packages\python310\site-packages (0.8.0)




[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\lucas\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
#import packages

#EDA / Visualization
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#data modeling
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import randint
from sklearn import tree
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier

#for github
import nbstripout

#for packaging model
import joblib

# EDA

In [6]:
#import training data and start exploratory data analysis
data = pd.read_csv('train.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

In [ ]:
data.head()

In [ ]:
data.columns

In [ ]:
data.count()

In [ ]:
nan_cols = data.columns[data.isna().any()]
null_col_values_focus = data[nan_cols].isna().mean() * 100

plt.figure(figsize=(15, 7))
sns.barplot(x=null_col_values_focus.index, y=null_col_values_focus.values)
plt.ylabel('Percentage of Missing Values (%)')
plt.title('Percentage of NaN Values by Column')
plt.show()

In [ ]:
data.isnull().sum()

### Data Visulization

In [ ]:
#number of depressed individuals and whether they are male or female and wether they are a working professional or are in schoool.

data_dep_vis = data[['Depression', 'Gender', 'Working Professional or Student']]

depressed_data = data_dep_vis[data_dep_vis['Depression'] == 1]


plt.figure(figsize=(10,6))
plt.bar(data=

In [ ]:
#age distribution of depressed individuals

data['Age'].describe()

In [ ]:
data_age_hist = data[data['Depression'] == 1]

plt.figure(figsize=(10, 6))
plt.hist(data_age_hist['Age'], bins=10, color='skyblue', edgecolor='black')
plt.xlabel("Age")
plt.ylabel("Frequency")
plt.title("Age Distribution of Depressed Individuals")
plt.show()

In [ ]:
#histogram of study / work hours for depressed individuals

data['Work/Study Hours'].describe()

In [ ]:
data_hours_hist = data[data['Depression'] == 1]

plt.figure(figsize=(10, 6))
plt.hist(data_hours_hist['Work/Study Hours'], bins=10, color='rebeccapurple', edgecolor='black')
plt.xlabel("Hours")
plt.ylabel("Frequency")
plt.title("Age Distribution of Depressed Individuals")
plt.show()

### Data Cleaning

In [ ]:
#When working with predictive modeling, you cannot use null values. There are 3 different approaches we can take to mitigate these values.
#The first option would be to completely wipe out all rows that contain at least 1 null value. This should be a last resort approach as we want to be 
#able to use as much data as possible for the model. The second option would be to find the mean or median of the column (if numeric) and fill in
#the nulls with that average value. This option is best if you have a large percentage of null values, but can throw off other statistics and models.
#Lastly, similarly with the average value method, we can fill in the nulls randomly. Even character values could be randomized if there is a unique set
#of different values. Again, this is not ideal unless there is a large portion of data missing. 

#Below we will determine the best route. 

In [ ]:
#profession unique values
data['Profession'].unique()

Based on the values here, we could easily fill in the 'nan' values with a random profession, but considering the context I plan to fill in the nan values with something like "unemployeed" or "no profession". 

In [ ]:
#academic pressure unique values
data['Academic Pressure'].unique()

Looks like the academic pressure may be a great depressive indicator, but we do have some nan values. Considering the context I believe we can fill in the 'nan' values with 0 as this would mean an individual has no academic pressure. 

In [ ]:
#work pressure unique values
data['Work Pressure'].unique()

Similary with the academic pressure, we should be able to replace those 'nan' values with 0.

In [ ]:
#Cumulative Grade Point Average unique values
data['CGPA'].unique()

The CGPA or Cumulative Grade Point Average is a good indicator of the students overall academic performance. 

In [ ]:
#Study Satisfaction unique values
data['Study Satisfaction'].unique()

In [ ]:
#Job Satisfaction unique values
data['Job Satisfaction'].unique()

In [ ]:
#Dietary habits unique values
data['Dietary Habits'].unique()

Based on these values, we will have to dig in more to discover the best route with this column or decide that it may not be useful. 

In [ ]:
#Degree unique values
data['Degree'].unique()

In [ ]:
#Degree unique value counts top
degree_nans = data['Degree'].value_counts(ascending=False)
degree_nans.head(10)

In [ ]:
#Degree unique value counts bottom
degree_nans = data['Degree'].value_counts(ascending=True)
degree_nans.head(10)

Due to the variety of degrees, assigning these unique ids for modeling will come in handy later on. 

In [ ]:
#Financial Stress unique values
data['Financial Stress'].unique()

Thankfully this stress indicator is farely clean and we should be able to replace the nan values with 0.

In [ ]:
#start here:
#work on cleaning out nan values across the board or figuring out which columns we want to use or "save" for modeling


#also next for cleaning will be trying to get the sleep hours out of the column 

In [ ]:
#sleep duration unique research
data['Sleep Duration'].unique()

In [ ]:
sleep_data = data['Sleep Duration'].value_counts(ascending=False)
sleep_data

Again, we have few options on how to deal with these values. In order to make them useful for modeling, we will need to normalize these values. The pro here is that there a bulk of the values are set by 4 different values. The con here is that another 32 values need to be parsed and assessed on how to properly assign a numeric value. 

I think the best approach to start would be to do "case when" for each value we see here and give it a best guess. Example 55-66 maybe just average to 60.5 hours or round down to 60. Or moderate we could do 6 hours? Some of the odd-ball string values we can just put in an average. 